In [1]:
import os
import numpy as np
import pandas as pd
import torch
from datasets import Dataset, DatasetDict, ClassLabel
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, f1_score, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
)

c:\Users\Aditya\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def load_data():
    arrow_path = "dataset/train/train/data-00000-of-00001.arrow"
    ds = Dataset.from_file(arrow_path)
    df = ds.to_pandas()

    df["text"] = (
        "Question: " + df["question"] +
        " [SEP] Answer: " + df["interview_answer"]
    )
    return df


df = load_data()
df.head()

,question,interview_question,interview_answer,label,url,inaudible,multiple_questions,affirmative_questions,text
0,How would you respond to the accusation that t...,Q. Of the Biden administration. And accused th...,"Well, look, first of all, theI am sincere abou...",Explicit,https://www.presidency.ucsb.edu/documents/the-...,False,False,False,Question: How would you respond to the accusat...
1,Do you think President Xi is being sincere abo...,Q. Of the Biden administration. And accused th...,"Well, look, first of all, theI am sincere abou...",General,https://www.presidency.ucsb.edu/documents/the-...,False,False,False,Question: Do you think President Xi is being s...
2,Do you believe the country's slowdown and gro...,Q. No worries. Do you believe the country's sl...,"Look, I think China has a difficult economic p...",Partial/half-answer,https://www.presidency.ucsb.edu/documents/the-...,False,False,False,Question: Do you believe the country's slowdo...
3,Are you worried about the meeting between Pre...,Q. No worries. Do you believe the country's sl...,"Look, I think China has a difficult economic p...",Dodging,https://www.presidency.ucsb.edu/documents/the-...,False,False,False,Question: Are you worried about the meeting b...
4,Is the President's engagement with Asian coun...,"Q. I can imagine. It is evening, I'd like to r...","Well, I hope I get to see Mr. Xi sooner than l...",Explicit,https://www.presidency.ucsb.edu/documents/the-...,False,False,False,Question: Is the President's engagement with ...


In [3]:
labels = sorted(df["label"].unique())
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}
df["label_id"] = df["label"].map(label2id)

dataset = Dataset.from_pandas(
    df[["text", "label_id"]].rename(columns={"label_id": "labels"}),
    preserve_index=False
)

features = dataset.features.copy()
features["labels"] = ClassLabel(names=labels)
dataset = dataset.cast(features)

Casting the dataset:   0%|          | 0/3448 [00:00<?, ? examples/s]

In [ ]:
split1 = dataset.train_test_split(
    test_size=0.2, seed=42, stratify_by_column="labels")
split2 = split1["test"].train_test_split(
    test_size=0.5, seed=42, stratify_by_column="labels")

datasets = DatasetDict({
    "train": split1["train"],
    "val": split2["train"],
    "test": split2["test"],
})

: 

In [ ]:
MODEL_NAME = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=512,
        padding=False
    )

tokenized = datasets.map(tokenize, batched=True, remove_columns=["text"])